Created on Fri Jan 17 17:03:35 2019  
Group 9  
@authors:  

In [1]:
import pandas as pd
import numpy as np

In [2]:
%run constants.ipynb

**let's load the data**

In [3]:
# Load Twitter Dataset
data = pd.read_excel(root+'Interpromo2020/Interpromo2020/TWITTER.xlsx')

In [4]:
# Load Comments from the file ALl Data
data1 = pd.read_excel(
    root+'Interpromo2020/Interpromo2020/All Data/ALL_DATA.xlsx')

In [5]:
data.head()

,contains_photo,contains_video,created_at,entities,favorite_count,is_quote_status,lang,quoted_status,quoted_status_id_str,retweet_count,text,user
0,1,0,2019-04-02 14:38:26,"{'hashtags': ['airBaltic'], 'symbols': [], 'us...",72,0,en,NaN,NaN,13,Good news for passengers on the Prague route! ...,"{'id_str': '26726695', 'name': 'airBaltic', 's..."
1,1,0,2019-04-01 19:00:01,"{'hashtags': ['Airbus', 'A220', 'avgeek', 'Afr...",111,0,en,NaN,NaN,19,"Thanks to the GTF, Mount Kilimanjaro 🗻 was bre...","{'id_str': '20814874', 'name': 'Pratt & Whitne..."
2,1,0,2019-04-01 10:40:21,"{'hashtags': ['A220'], 'symbols': [], 'user_me...",29,0,en,NaN,NaN,6,"Sunbathing in Bucharest: YL-CSM, @Airbus #A220...","{'id_str': '1403040254', 'name': 'Aeronews', '..."
3,1,0,2019-03-31 10:20:23,"{'hashtags': [], 'symbols': [], 'user_mentions...",74,0,en,NaN,NaN,9,Today we opened long-awaited @airBaltic route ...,"{'id_str': '52174696', 'name': 'Martin Gauss',..."
4,1,0,2019-03-31 06:10:27,"{'hashtags': ['airBaltic'], 'symbols': [], 'us...",58,0,en,NaN,NaN,9,The new #airBaltic route between Riga and Dubl...,"{'id_str': '26726695', 'name': 'airBaltic', 's..."


In [6]:
data1.head()

,Data_Source,Airline_Name,Airline_Type,Region_Operation,Aircraft_Type,Cabin_Class,Type_Of_Lounge,Type_Of_Traveller,Date_Visit,Date_Flown,...,Seat_Recline,Seat_Width,Washrooms,Value_For_Money,Overall_Customer_Rating,Overall_Service_Rating,Overall_Airline_Rating,Recommended,Date_Review,Review
0,Airline Quality Airline,ab-aviation,NaN,NaN,E120,Economy Class,NaN,Solo Leisure,NaN,June 2019,...,NaN,NaN,NaN,2.0,1,NaN,NaN,no,25th June 2019,✅ Trip Verified | Moroni to Anjouan. It is a v...
1,Airline Quality Airline,ab-aviation,NaN,NaN,Embraer E120,Economy Class,NaN,Solo Leisure,NaN,June 2019,...,NaN,NaN,NaN,2.0,1,NaN,NaN,no,25th June 2019,✅ Trip Verified | Anjouan to Dzaoudzi. A very...
2,Airline Quality Airline,adria-airways,NaN,NaN,NaN,Economy Class,NaN,Solo Leisure,NaN,September 2019,...,NaN,NaN,NaN,1.0,1,NaN,NaN,no,28th September 2019,Not Verified | Please do a favor yourself and...
3,Airline Quality Airline,adria-airways,NaN,NaN,NaN,Economy Class,NaN,Couple Leisure,NaN,September 2019,...,NaN,NaN,NaN,1.0,1,NaN,NaN,no,24th September 2019,✅ Trip Verified | Do not book a flight with th...
4,Airline Quality Airline,adria-airways,NaN,NaN,CR 900,Economy Class,NaN,Couple Leisure,NaN,September 2019,...,NaN,NaN,NaN,1.0,1,NaN,NaN,no,17th September 2019,✅ Trip Verified | Had very bad experience wit...


In [14]:
# Let's see how does a comment look like
data['text'].values[10]

'The 1st production employees for A220 production in Mobile AL have today begun their new hire orientation training at AIDT, and after a month of local training will transition to Mirabel, Quebec to train for 3 months in the A220 assembly line there. https://t.co/6O8fK3VBv6'

In [15]:
def sentense2cleanTokens(sent):
    """This functions takes as input a tweet or comment and does 
    the standardization
    
    parameters :
        sent : Tweet or comment
    Out:
        sent: Standardized tweet/comment

    """
    sent = "".join(
        [x if (x.isalpha() or x.isnumeric()) else " " for x in sent])
    sent = " ".join(sent.split())
    return sent

In [16]:
sentense2cleanTokens(data['text'].values[3])

'Today we opened long awaited airBaltic route between Riga and Dublin Good to see so many happy passengers boarding our Airbus A220 aircraft Thank you IrelandEmbLat and DublinAirport for great cooperation https t co fHiCkxsUrm'

In [17]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
from gensim.models import Word2Vec
from nltk.corpus import wordnet as wn

In [18]:
model = gensim.models.KeyedVectors.load_word2vec_format(
    pathword2vec, binary=True)

In [19]:
def get_two_words(listwords):
    """This function takes as input the list of the ten words that 
    are most similar to a contextual representation of a tweet and 
    returns the the two most similar words of the list
    
    parameters :
        sent:  list of words
    Out:
        Set of two most similar words
    """
    data = []
    for w1 in listwords:
        data.append([w1, w1, 0])
        for w2 in listwords:
            if (w1.lower() != w2.lower()):
                w1s = wn.synsets(w1)[0]
                w2s = wn.synsets(w2)[0]
                if (w1s.path_similarity(w2s) == None):
                    data.append([w1, w2, 0])
                else:
                    data.append([w1, w2, w1s.path_similarity(w2s)])
    df = pd.DataFrame(data, columns=['w1', 'w2', 'sim'])
    i = np.argmax(df["sim"].values)
    return(wn.synsets(df.iloc[i]['w1'])[0], 
           wn.synsets(df.iloc[i]['w2'])[0])

In [20]:
get_two_words(['France','Paris','Toulouse'])

(Synset('paris.n.01'), Synset('toulouse.n.01'))

In [21]:
def find_hypernyms(com, model):
    """This functions takes as input a tweet or comment and the word2vec 
    model and returns for each sentence of a tweet/comment the common 
    hypernym of the two most similar words obtained from the previous function

    parameters :
        com:  Tweet or comment
        model: pretrained word embedding model
    Out:
        listentities: List of the common hypernyms for each sentence of a tweet/comment
    """
    listtext = com.split('.')
    listentities = []
    filtered = []
    for text in listtext:
        text = sentense2cleanTokens(text)
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(text)
        filtered_sentence = [w for w in word_tokens 
                             if not w in stop_words]
        filtered.append(filtered_sentence)
        context = np.sum(
            [model[word] for word in filtered_sentence 
             if word in (model.vocab)], axis=0)
        if context != []:
            contexts = model.similar_by_vector(
                context, topn=10, restrict_vocab=None)
            most_similar_words = [x[0] for x in contexts]
            most_similar_words = [
                x for x in most_similar_words if wn.synsets(x) != []]
            first_word, second_word = get_two_words(most_similar_words)
            hypernym = first_word.lowest_common_hypernyms(second_word)
            #hypernym =[]
            if hypernym != []:
                listentities.append(hypernym[0].lemmas()[0].name())
    if listentities == []:
        for x in filtered:
            for j in x:
                listentities.append(j)
    return(listentities)

In [22]:
def find_context(com,model,intents):
    """This functions takes as input a tweet or comment, the word2vec 
    model and a list of intents we want to detect and returns for each 
    tweet/comment the intent based on the wordnet ressource

    parameters :
        com:  Tweet or comment
        model: pretrained word embedding model
        intents: pre-defined list of intents
    Out:
        Intent: The intent the tweet/comment talks about
    """
    hypernyms = find_hypernyms(com,model)
    hyp = hypernyms[0]
    score = 0 
    for h in hypernyms:
        for i in intents:
            if wn.synsets(h)!= []:
                w1s = wn.synsets(h)[0]
                w2s = wn.synsets(i)[0]
                sc = w1s.path_similarity(w2s)
                if ((sc!= None) and (sc> score)):
                    hyp = h 
                    score = sc
    return (hyp)

In [23]:
data1['Review'].values[14]

'✅ Trip Verified |  Frankfurt to Ljubljana. Flight was very comfortable, professional crew. The monitors above passengers provide informations about flight (speed, altitude, temperature, time to landing, etc.)'

In [24]:
find_hypernyms(
    "I have this thing for airplane food, I really love it. It doesn't\
    matter how it tastes, but it's all packaged up and breaks up the\
    monotony of the flight. The internet is slow tho",
    model)

/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


['abstraction', 'psychological_feature']

In [25]:
find_context(data1['Review'].values[14],model,['food','person','flight','seat'])

/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


'pilot'

In [26]:
####find out if spacy does the wtcenter 

In [27]:
from scipy import spatial

In [28]:
def find_context_w2v(com,model,intents):
    """This functions takes as input a tweet or comment, the word2vec model 
    and a list of intents we want to detect and returns for each tweet/comment
    the intent based on the word2vec similarities. It does the same treatment
    as the previous one but is more accurate. That's so the method we'll keep
    to detect an intent from a tweet/comment

    parameters :
        com:  Tweet or comment
        model: pretrained word embedding model
        intents: pre-defined list of intents
    Out:
        Intent: The intent the tweet/comment talks about
    """
    hypernyms = find_hypernyms(com,model)
    hyp = hypernyms[0]
    score = 0 
    for h in hypernyms:
        for i in intents:
            sc = np.mean([model.similarity(k,i) for k in h.split('_') if k in model.wv.vocab])
            if ((sc!= None) and (sc> score)):
                hyp = i 
                score = sc
    return (hyp)

In [29]:
find_context_w2v(
    "I got upgraded to Premium Economy by @airfrance on flight\
    AF0814 from Paris to Nairobi. Wider leg room, seat recline, privacy, wider seat,\
    fine food👌Thank you AirFrance🥂",
    model,
    ['space', 'food', 'staff', 'late', 'good', 'bad'])

/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


'space'

In [30]:
find_context_w2v(
    "travelled on domestic US and Air France International\
    flight with carry on luggage. Last flight leg internal in France was made\
    to check in my carry on! Aircraft was A320 loads of cabin space, not even\
    close to full!!!! Makes no sense.",
    model,
    ['space', 'food', 'staff', 'late', 'good', 'bad'])

/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


'space'

In [31]:
find_context_w2v(
    "Terrible customer service @Ryanair. Website would\
    not process payment for flights last night due to awful website\
    glitch and now price has increased by £100. Phoned customer service\
    who said they couldn’t help. Total lack of customer support",
    model,
    ['space', 'food', 'staff', 'late', 'good', 'bad'])

/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


'staff'